In [102]:
import os
import pandas as pd
import string
import unicodedata
import re

folder_path = 'Forms/AV1'
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [103]:
def preprocess_column(list_):

    p = list()

    for el in list_:

        v = str(el).translate(str.maketrans('', '', string.punctuation)).lower()
        v = unicodedata.normalize('NFD', v)
        v = ''.join(c for c in v if unicodedata.category(c) != 'Mn')

        v = v.strip()
        v = re.sub(r'\s+', ' ', v)

        p.append(v)

    return p

In [104]:
def preprocess_cpf(list_):

    p = list()

    for el in list_:

        v = str(el).translate(str.maketrans('', '', string.punctuation)).lower()

        p.append(v)

    return p

In [105]:
def preprocess_nome(list_):

    p = list()

    for el in list_:

        v = str(el).translate(str.maketrans('', '', string.punctuation)).upper()
        v = unicodedata.normalize('NFD', v)
        v = ''.join(c for c in v if unicodedata.category(c) != 'Mn')

        v = v.strip()
        v = re.sub(r'\s+', ' ', v)

        p.append(v)

    return p

In [106]:
list_df = list()

for file in files:

    df = pd.read_excel(f"{folder_path}/{file}")
    df.columns = preprocess_column(list(df.columns))
    cpf_column = [x for x in list(df.columns) if "cpf" in x]
    assert len(cpf_column) == 1
    df["cpf"] = preprocess_cpf(list(df[cpf_column[0]]))
    df["cpf"] = ["0" + el if len(el)==10 else el for el in list(df["cpf"])]

    name_column = [x for x in list(df.columns) if "seu nome" in x]
    assert len(name_column) == 1
    df[f"{file}_nome"] = preprocess_nome(list(df[name_column[0]]))

    pont_column = [x for x in list(df.columns) if "pontuacao" in x]
    assert len(pont_column) == 1
    df[f"{file}_pontuacao"] = preprocess_column(list(df[pont_column[0]]))

    df = df[["cpf", f"{file}_pontuacao", f"{file}_nome"]]

    list_df.append(df)

In [107]:
out_df = list_df[0]

# Merge the remaining DataFrames on the 'cpf' column
for df in list_df[1:]:
    # Merge using outer join on 'cpf' to keep all unique cpfs
    out_df = pd.merge(out_df, df, on='cpf', how='outer')

out_df.fillna("", inplace=True)

In [108]:
out_df = out_df.groupby('cpf').agg(list)
out_df.reset_index(inplace=True)

In [109]:
nome_columns = [col for col in out_df.columns if col.endswith('_nome')]

out_df['nome'] = out_df[nome_columns].apply(
    lambda row: list({item for sublist in row for item in sublist}), axis=1
)

In [110]:
for n in nome_columns:

    del out_df[n]

In [112]:
out_df.to_excel("results.xlsx", index=None)